In [4]:
import pandas as pd

In [5]:
dataset = pd.read_csv('data/processed/openhust_with_id.csv')

In [6]:
dataset.head()

,ID,Document,Summary,Dataset
0,1,"Theo đó , các sở , ngành trên địa bàn phải vào...","Các quận , huyện , thị xã tuyên truyền bằng nh...",vietnews
1,2,"Chiều 12/3 , ông Vũ Hùng Triều , Trưởng phòng ...",Cháu bé được phát hiện trong tình trạng chưa c...,vietnews
2,3,"Thoạt đầu nhìn vào bức ảnh , nếu không có dấu ...",Báo tuyết ( Panthera uncia ) được mệnh danh là...,vietnews
3,4,"Khoảng 9h30 sáng 5/6 , anh Đào Nhật Tuấn ( 30 ...","Thấy con chim bị mắc bẫy rơi xuống hồ nước , a...",vietnews
4,5,Liên minh Tự do Dân sự Mỹ ( ACLU ) đã nhận đượ...,"Loại "" huyết thanh nói thật "" mà CIA sử dụng đ...",vietnews


In [8]:
dataset.rename(columns={'Document': 'Text'}, inplace=True)


In [9]:
dataset = dataset[['Summary','Text']]
dataset = dataset.head(10000)

In [10]:
dataset.head()

,Summary,Text
0,"Các quận , huyện , thị xã tuyên truyền bằng nh...","Theo đó , các sở , ngành trên địa bàn phải vào..."
1,Cháu bé được phát hiện trong tình trạng chưa c...,"Chiều 12/3 , ông Vũ Hùng Triều , Trưởng phòng ..."
2,Báo tuyết ( Panthera uncia ) được mệnh danh là...,"Thoạt đầu nhìn vào bức ảnh , nếu không có dấu ..."
3,"Thấy con chim bị mắc bẫy rơi xuống hồ nước , a...","Khoảng 9h30 sáng 5/6 , anh Đào Nhật Tuấn ( 30 ..."
4,"Loại "" huyết thanh nói thật "" mà CIA sử dụng đ...",Liên minh Tự do Dân sự Mỹ ( ACLU ) đã nhận đượ...


In [11]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)

In [12]:
full_dataset = dataset.train_test_split(test_size=0.2, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['Summary', 'Text'],
    num_rows: 8000
})
Dataset({
    features: ['Summary', 'Text'],
    num_rows: 2000
})


In [13]:
import torch
import pprint
import evaluate
import numpy as np

In [14]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

In [18]:
MODEL = 't5-small'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 5
OUT_DIR = './result111'
MAX_LENGTH = 512 

In [19]:
tokenizer = T5Tokenizer('models/trained_spiece1.model')

In [20]:
from functools import partial
from transformers import T5Tokenizer

# Initialize the tokenizer
# tokenizer = T5Tokenizer.from_pretrained(MODEL)
tokenizer = T5Tokenizer('models/trained_spiece1.model')
# Function to convert text data into model inputs and targets
def preprocess_function(examples, tokenizer):
    inputs = [f"summarize: {text}" for text in examples['Text']]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summary']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create a partial function with the tokenizer
preprocess_function_with_tokenizer = partial(preprocess_function, tokenizer=tokenizer)

# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)


Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [22]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 total parameters.
60,506,624 training parameters.


In [23]:
rouge = evaluate.load("rouge")
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [24]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [25]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS, # EPOCHS = 4
    per_device_train_batch_size=BATCH_SIZE, # BATCH_SIZE = 4
    per_device_eval_batch_size=BATCH_SIZE, 
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=400,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)
 
history = trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]

{'loss': 19.0319, 'grad_norm': 135.919921875, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 18.9736, 'grad_norm': 127.6744384765625, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.01}
{'loss': 17.118, 'grad_norm': 191.77078247070312, 'learning_rate': 6e-06, 'epoch': 0.01}
{'loss': 16.1299, 'grad_norm': 190.88998413085938, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.02}
{'loss': 14.6072, 'grad_norm': 348.1108093261719, 'learning_rate': 1e-05, 'epoch': 0.03}
{'loss': 11.8206, 'grad_norm': 107.14894104003906, 'learning_rate': 1.2e-05, 'epoch': 0.03}
{'loss': 9.5774, 'grad_norm': 469.3350830078125, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.04}
{'loss': 7.5948, 'grad_norm': 115.38677978515625, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.04}
{'loss': 5.5817, 'grad_norm': 80.26242065429688, 'learning_rate': 1.8e-05, 'epoch': 0.04}
{'loss': 3.299, 'grad_norm': 20.27674102783203, 'learning_rate': 2e-05, 'epoch': 0.05}
{'loss': 1.9197, 'grad_norm':

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 0.532606840133667, 'eval_rouge1': 0.5353, 'eval_rouge2': 0.2722, 'eval_rougeL': 0.4167, 'eval_gen_len': 41.1765, 'eval_runtime': 105.4431, 'eval_samples_per_second': 18.968, 'eval_steps_per_second': 4.742, 'epoch': 0.2}
{'loss': 0.5712, 'grad_norm': 0.539078414440155, 'learning_rate': 8.2e-05, 'epoch': 0.2}
{'loss': 0.5143, 'grad_norm': 0.577938437461853, 'learning_rate': 8.4e-05, 'epoch': 0.21}
{'loss': 0.4925, 'grad_norm': 0.43397706747055054, 'learning_rate': 8.6e-05, 'epoch': 0.21}
{'loss': 0.5596, 'grad_norm': 0.3756245970726013, 'learning_rate': 8.800000000000001e-05, 'epoch': 0.22}
{'loss': 0.5574, 'grad_norm': 0.4262297749519348, 'learning_rate': 9e-05, 'epoch': 0.23}
{'loss': 0.5493, 'grad_norm': 0.41229790449142456, 'learning_rate': 9.200000000000001e-05, 'epoch': 0.23}
{'loss': 0.5184, 'grad_norm': 0.36312904953956604, 'learning_rate': 9.4e-05, 'epoch': 0.23}
{'loss': 0.5616, 'grad_norm': 0.39506882429122925, 'learning_rate': 9.6e-05, 'epoch': 0.24}
{'loss': 0.

  0%|          | 0/500 [00:00<?, ?it/s]

...

{'eval_loss': 0.4218143820762634, 'eval_rouge1': 0.5853, 'eval_rouge2': 0.3151, 'eval_rougeL': 0.4599, 'eval_gen_len': 42.18, 'eval_runtime': 94.6954, 'eval_samples_per_second': 21.12, 'eval_steps_per_second': 5.28, 'epoch': 4.6}
{'loss': 0.4575, 'grad_norm': 0.30436521768569946, 'learning_rate': 8.315789473684212e-06, 'epoch': 4.61}
{'loss': 0.5398, 'grad_norm': 0.362328439950943, 'learning_rate': 8.210526315789475e-06, 'epoch': 4.61}
{'loss': 0.4093, 'grad_norm': 0.352314829826355, 'learning_rate': 8.105263157894736e-06, 'epoch': 4.62}
{'loss': 0.4403, 'grad_norm': 0.39526820182800293, 'learning_rate': 8.000000000000001e-06, 'epoch': 4.62}
{'loss': 0.4777, 'grad_norm': 0.35252729058265686, 'learning_rate': 7.894736842105263e-06, 'epoch': 4.62}
{'loss': 0.4104, 'grad_norm': 0.43686583638191223, 'learning_rate': 7.789473684210528e-06, 'epoch': 4.63}
{'loss': 0.4028, 'grad_norm': 0.31673651933670044, 'learning_rate': 7.68421052631579e-06, 'epoch': 4.63}
{'loss': 0.3903, 'grad_norm': 0.4

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 0.42202457785606384, 'eval_rouge1': 0.587, 'eval_rouge2': 0.316, 'eval_rougeL': 0.4604, 'eval_gen_len': 42.18, 'eval_runtime': 98.9272, 'eval_samples_per_second': 20.217, 'eval_steps_per_second': 5.054, 'epoch': 4.8}
{'loss': 0.476, 'grad_norm': 0.34321388602256775, 'learning_rate': 4.105263157894737e-06, 'epoch': 4.8}
{'loss': 0.4128, 'grad_norm': 0.31285735964775085, 'learning_rate': 4.000000000000001e-06, 'epoch': 4.81}
{'loss': 0.4246, 'grad_norm': 0.23508596420288086, 'learning_rate': 3.894736842105264e-06, 'epoch': 4.81}
{'loss': 0.3836, 'grad_norm': 0.34628164768218994, 'learning_rate': 3.789473684210527e-06, 'epoch': 4.82}
{'loss': 0.4143, 'grad_norm': 0.3116768002510071, 'learning_rate': 3.6842105263157892e-06, 'epoch': 4.83}
{'loss': 0.5326, 'grad_norm': 0.44306761026382446, 'learning_rate': 3.5789473684210525e-06, 'epoch': 4.83}
{'loss': 0.3877, 'grad_norm': 0.5364938974380493, 'learning_rate': 3.4736842105263158e-06, 'epoch': 4.83}
{'loss': 0.3931, 'grad_norm'

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 0.42163577675819397, 'eval_rouge1': 0.5859, 'eval_rouge2': 0.3158, 'eval_rougeL': 0.4604, 'eval_gen_len': 42.18, 'eval_runtime': 101.8673, 'eval_samples_per_second': 19.633, 'eval_steps_per_second': 4.908, 'epoch': 5.0}
{'train_runtime': 4633.6202, 'train_samples_per_second': 8.633, 'train_steps_per_second': 2.158, 'train_loss': 0.5817438374757766, 'epoch': 5.0}


In [27]:
model_path = f"{OUT_DIR}/checkpoint-10000"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer('models/trained_spiece1.model')

In [28]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=200,
        num_beams=num_beams,
        # early_stopping=True,
    )

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [29]:
import json

# Đọc file JSON
with open('result111/checkpoint-10000/trainer_state.json', 'r') as file:
    data = json.load(file)

# Lấy từ điển có trường 'log_history'
log_history = data.get('log_history', [])


filtered_logs = [log for log in log_history if log.get('step', 0) % 400 == 0 and 'eval_rouge1' in log]

# In ra các từ điển đã lọc được
for log in filtered_logs:
    print(log)



{'epoch': 0.2, 'eval_gen_len': 41.1765, 'eval_loss': 0.532606840133667, 'eval_rouge1': 0.5353, 'eval_rouge2': 0.2722, 'eval_rougeL': 0.4167, 'eval_runtime': 105.4431, 'eval_samples_per_second': 18.968, 'eval_steps_per_second': 4.742, 'step': 400}
{'epoch': 0.4, 'eval_gen_len': 42.0995, 'eval_loss': 0.4979194104671478, 'eval_rouge1': 0.546, 'eval_rouge2': 0.2839, 'eval_rougeL': 0.4289, 'eval_runtime': 106.9859, 'eval_samples_per_second': 18.694, 'eval_steps_per_second': 4.674, 'step': 800}
{'epoch': 0.6, 'eval_gen_len': 42.178, 'eval_loss': 0.48442018032073975, 'eval_rouge1': 0.5454, 'eval_rouge2': 0.2871, 'eval_rougeL': 0.4322, 'eval_runtime': 102.8896, 'eval_samples_per_second': 19.438, 'eval_steps_per_second': 4.86, 'step': 1200}
{'epoch': 0.8, 'eval_gen_len': 42.1795, 'eval_loss': 0.4741900563240051, 'eval_rouge1': 0.5421, 'eval_rouge2': 0.2872, 'eval_rougeL': 0.4317, 'eval_runtime': 104.7672, 'eval_samples_per_second': 19.09, 'eval_steps_per_second': 4.772, 'step': 1600}
{'epoch': 

In [30]:
import pandas as pd

# Chuyển danh sách từ điển thành DataFrame
df = pd.DataFrame(filtered_logs)

# Tạo một bản sao của DataFrame
styled_df = df.style

# Tùy chỉnh màu sắc cho cột 'eval_rouge1'
styled_df = styled_df.apply(lambda row: ['color: blue' if val is None else '' for val in row], axis=1, subset=['eval_rouge1'])

# Hiển thị DataFrame đã tùy chỉnh với màu sắc
styled_df


,epoch,eval_gen_len,eval_loss,eval_rouge1,eval_rouge2,eval_rougeL,eval_runtime,eval_samples_per_second,eval_steps_per_second,step
0,0.200000,41.176500,0.532607,0.535300,0.272200,0.416700,105.443100,18.968000,4.742000,400
1,0.400000,42.099500,0.497919,0.546000,0.283900,0.428900,106.985900,18.694000,4.674000,800
2,0.600000,42.178000,0.484420,0.545400,0.287100,0.432200,102.889600,19.438000,4.860000,1200
3,0.800000,42.179500,0.474190,0.542100,0.287200,0.431700,104.767200,19.090000,4.772000,1600
4,1.000000,42.180000,0.468065,0.550500,0.290900,0.436600,105.802500,18.903000,4.726000,2000
5,1.200000,42.180000,0.461144,0.557800,0.296400,0.441600,105.789400,18.905000,4.726000,2400
6,1.400000,42.180000,0.457587,0.565300,0.300600,0.446900,103.978500,19.235000,4.809000,2800
7,1.600000,42.180000,0.452992,0.562900,0.301500,0.447600,102.586500,19.496000,4.874000,3200
8,1.800000,42.180000,0.448591,0.572000,0.305900,0.452000,101.457900,19.713000,4.928000,3600
9,2.000000,42.180000,0.443571,0.570000,0.305800,0.450700,100.842700,19.833000,4.958000,4000


In [36]:
import gradio as gr
 
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [44]:
def summarize_text(text):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=100,
        truncation=True,
        padding='max_length'
    )
 
    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=100,
        num_beams=5,
        # early_stopping=True
    )
 
    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [45]:
interface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(lines=10, placeholder='Enter Text Here...', label='Input text'),
    outputs=gr.Textbox(label='Summarized Text'),
    title='Text Summarizer using T5'
)
interface.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
